In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import pickle

# fondation 

Essential to run all this section: load the pickl of our experiment (think to change the path ;)  
You can print level on and level two keys in case you have a doubt  (cell 5) one can use these list to loop over the dataset  
- Level 1 will let you chose which method you want to see: raw, or method 1,2, or 3  (corresponding to each challenge)
- level 2 which of the 6 sequence  


In [2]:
def load_dict_of_dicts_pickle(filename):

    with open(filename, 'rb') as f:
        return pickle.load(f)

In [3]:
data = load_dict_of_dicts_pickle(r"C:\Users\WillQuantique\OneDrive - univ-lyon2.fr\Fac\M1\DataChallege\git\M1_S8_dataChallenge_24\code\test_jupyter_notebook\data_for_metrics.pkl")

In [4]:
level_one_keys = list(data.keys())
level_two_keys =[]
for k in level_one_keys:
    level_two_keys.extend(list(data[k].keys()))
level_two_keys = set(level_two_keys)

In [5]:
print(level_one_keys)
print(level_two_keys)

['method_1', 'method_2', 'method_3', 'raw']
{'car_scene_2', 'scene_1', 'car_scene_3', 'scene_3', 'car_scene_1', 'scene_2'}


# building blocks
Here you will find the bulding blocks for the autoadjustment functions  
Don't hesitate to build your own (you may add new metrics function here to help fo r that matter)

In [ ]:
def sorted_uniform_pictures(arr):
    """
    Sort (t, x, y) array's t slices from most uniform to least uniform.

    Parameters:
    arr (np.ndarray): Input 3D array of shape (t, x, y).

    Returns:
    np.ndarray: A 3D array where the t slices are sorted by uniformity.
    """
    t, x, y = arr.shape
    
    # Compute the variance for each t slice
    variances = np.var(arr, axis=(1, 2))
    
    # Get the sorted indices based on variances
    sorted_indices = np.argsort(variances)
    
    # Extract and sort the t slices
    sorted_slices = arr[sorted_indices]
    
    return sorted_slices

In [ ]:

def save_video_from_images(images, output_filename, fps=60):
    """
    Save a sequence of images as a video.

    Parameters:
    images (list or array): List or array of images (each image should be a 2D or 3D NumPy array).
    output_filename (str): Output video filename (e.g., 'output.avi').
    fps (int): Frames per second for the video.
    """
    # Get the dimensions of the images
    height, width = images[0].shape[:2]

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_filename, fourcc, fps, (width, height))

    for img in images:
        # Ensure image is in the correct format (uint8)
        img_uint8 = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
        # If the image is grayscale, convert it to BGR
        if len(img_uint8.shape) == 2:
            img_uint8 = cv2.cvtColor(img_uint8, cv2.COLOR_GRAY2BGR)
        out.write(img_uint8)

    # Release everything when the job is finished
    out.release()

In [ ]:
def adjust_luminosity_contrast(image: np.ndarray, alpha: float, beta: int) -> np.ndarray:
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted_image

In [ ]:
def count_features(image: np.ndarray) -> int:
    orb = cv2.ORB_create()
    if image is None or image.size == 0 or len(image.shape) != 2:
        return 0
    keypoints = orb.detect(image, None)
    return len(keypoints)

In [ ]:
def slice_with_least_features(array):
    orb = cv2.ORB_create()
    feature_counts = []
    
    for t in range(array.shape[0]):
        slice_t = array[t]
        slice_t = slice_t.astype(np.uint8)
        keypoints = orb.detect(slice_t, None)
        feature_counts.append(len(keypoints))
    
    min_features_idx = np.argmin(feature_counts)
    return array[min_features_idx]

In [ ]:
def count_and_check_threshold(array, target_value: float, x: float):
    total_values = array.size
    target_value_count = np.count_nonzero(array >= target_value)
    percentage = (target_value_count / total_values) * 100
    return percentage < x

## autoadjust 1
use feature detection as a metric for autoadjustment

In [ ]:
def auto_adjust_luminosity_contrast(image: np.ndarray) -> np.ndarray:
    best_feature_count = -1
    best_image = None
    best_alpha = 0.01
    best_beta = 0
    if len(image.shape)==3:
        image = sorted_uniform_pictures(image)[int(len(image)/2)]
    print(image.shape)
    alpha_range = np.arange(0.0, 1.5, 0.005)
    beta_range = np.arange(0, 100, 5)

    
    for beta in beta_range:
        for alpha in alpha_range:
            adjusted_image = adjust_luminosity_contrast(image, alpha, beta)
            if adjusted_image is None or adjusted_image.size == 0 or np.isnan(adjusted_image).any():
                continue
            current_feature_count = count_features(adjusted_image)

            if current_feature_count > best_feature_count:
                if count_and_check_threshold(adjusted_image, 255, 99):
                    best_feature_count = current_feature_count
                    best_image = adjusted_image
                    best_alpha = alpha
                    best_beta = beta
                else:
                    break

    if best_image is None:
        raise ValueError("No valid image was found. Please check the input image and parameters.")
    
    print(f"Best alpha: {best_alpha}, Best beta: {best_beta}")
    return best_image, best_alpha, best_beta

# Auto adjust 2
Uses standard deviation as a metric to find a sweet spot for adjusting the contrast  
Then uses the mean as a metric to adjust for luminosity.  
The margine of error account for 256 possible pixel values.

In [ ]:
def auto_adjust_luminosity_contrast_2(image: np.ndarray) -> np.ndarray:
    best_image = None
    best_alpha = 0.01
    best_beta = 0
    
    if len(image.shape) == 3:
        image = image[int(len(image) / 2)]
    
    alpha_range = np.arange(0.0, 2.0, 0.005)
    beta_range = np.arange(-100, 300, 5)

    # Find best alpha based on standard deviation
    best_metric = -1
    for alpha in alpha_range:
        adjusted_image = adjust_luminosity_contrast(image, alpha, 0)
        if adjusted_image is None or adjusted_image.size == 0 or np.isnan(adjusted_image).any():
            continue
        
        current_std = np.std(adjusted_image)
        
        if current_std > best_metric and 10 < current_std < 100:
            best_metric = current_std
            best_alpha = alpha
            best_image = adjusted_image
    
    if best_image is None:
        raise ValueError("No valid image was found in the first pass. Please check the input image and parameters.")
    
    # Now, find the best beta based on mean brightness
    best_metric = -1
    final_best_image = None
    for beta in beta_range:
        adjusted_image = adjust_luminosity_contrast(image, best_alpha, beta)
        if adjusted_image is None or adjusted_image.size == 0 or np.isnan(adjusted_image).any():
            continue
        
        current_mean_brightness = np.mean(adjusted_image)
        
        if current_mean_brightness > best_metric and 50 < current_mean_brightness < 200:
            best_metric = current_mean_brightness
            best_beta = beta
            final_best_image = adjusted_image
    
    if final_best_image is None:
        raise ValueError("No valid image was found in the second pass. Please check the input image and parameters.")
    
    return final_best_image, best_alpha/10, best_beta/2

# run 

This is the part where you can make a mess and explore the data at wil ;)